In [5]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d richlukich/itlp-campus-dataset

import zipfile
import os

with zipfile.ZipFile('itlp-campus-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('itlp-campus-dataset')

os.listdir('itlp-campus-dataset')

Saving kaggle (1).json to kaggle (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/richlukich/itlp-campus-dataset
License(s): MIT
100% 4.06G/4.07G [00:56<00:00, 79.5MB/s]
100% 4.07G/4.07G [00:56<00:00, 76.7MB/s]


['public']

In [2]:
#@title
import pandas as pd
from pathlib import Path
import numpy as np
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import shapely as shp
from shapely.wkt import loads
from pprint import pprint
import json
from shapely.geometry import Point
import folium

In [3]:
data = {
    'Name': ["Main Building", "University Park", "Dormitory #1", "Sports Complex", "University Cafe"],
    'Type': ["Main", "Park", "Dormitory", "Sport", "Cafe"],
    'Longitude': [37.518434, 37.534927 , 37.521929, 37.522962, 37.520671],
    'Latitude': [55.929444, 55.921612, 55.930161, 55.929596, 55.930630]
}
df = pd.DataFrame(data)
df.to_csv('university_geodata.csv', index=False)


In [4]:



#df = pd.read_csv('university_geodata.csv')
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.set_crs(epsg=4326, inplace=True)

print(gdf)


              Name       Type  Longitude   Latitude                   geometry
0    Main Building       Main  37.518434  55.929444  POINT (37.51843 55.92944)
1  University Park       Park  37.534927  55.921612  POINT (37.53493 55.92161)
2     Dormitory #1  Dormitory  37.521929  55.930161  POINT (37.52193 55.93016)
3   Sports Complex      Sport  37.522962  55.929596  POINT (37.52296 55.92960)
4  University Cafe       Cafe  37.520671  55.930630  POINT (37.52067 55.93063)


In [6]:
DATA_ROOT = '/content/itlp-campus-dataset/public'
train_df = pd.read_csv(DATA_ROOT + '/train.csv').drop(columns='Unnamed: 0')

In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

df = pd.read_csv('university_geodata.csv')

geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

gdf = gpd.GeoDataFrame(df, geometry=geometry)

gdf.set_crs(epsg=4326, inplace=True)

m = folium.Map(location=[gdf['Latitude'].mean(), gdf['Longitude'].mean()], zoom_start=15)

for idx, row in gdf.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        tooltip=row['Type']
    ).add_to(m)

m.save('university_map.html')
m


In [12]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

file_path = '/content/train.csv'
df = pd.read_csv(file_path)

df_cleaned = df.drop(columns=['Unnamed: 0'])

start_lat = 55.929257
start_lon = 37.518069

# 1 градус широты ≈ 111 км, 1 градус долготы ≈ 111 км * cos(широта)
def convert_to_lat_lon(start_lat, start_lon, tx, ty):
    delta_lat = ty / 111000  # Метры в градусы широты
    delta_lon = tx / (111000 * np.cos(np.radians(start_lat)))  # Метры в градусы долготы
    return start_lat + delta_lat, start_lon + delta_lon

df_cleaned['Latitude'], df_cleaned['Longitude'] = zip(*df_cleaned.apply(
    lambda row: convert_to_lat_lon(start_lat, start_lon, row['tx'], row['ty']), axis=1))

geometry = [Point(xy) for xy in zip(df_cleaned['Longitude'], df_cleaned['Latitude'])]

gdf = gpd.GeoDataFrame(df_cleaned, geometry=geometry)

gdf.set_crs(epsg=4326, inplace=True)

m = folium.Map(location=[start_lat, start_lon], zoom_start=16)

for idx, row in gdf.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['track'],
        tooltip=f"Timestamp: {row['timestamp']}"
    ).add_to(m)

m.save('robot_movement_map.html')
m

